<a href="https://colab.research.google.com/github/Einstien1/Movir-recomendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/MOvie DAta/wiki_movie_plots_deduped.csv')
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [ ]:
len(data)

34886

In [ ]:
import numpy as np
np.unique(data['Origin/Ethnicity'])
  
len(data.loc[data['Origin/Ethnicity']=='American'])


17377

In [ ]:
len(data.loc[data['Origin/Ethnicity']=='British'])

3670

In [ ]:
df1 = pd.DataFrame(data.loc[data['Origin/Ethnicity']=='American'])
df2 = pd.DataFrame(data.loc[data['Origin/Ethnicity']=='British'])
data = pd.concat([df1, df2], ignore_index = True)

In [ ]:
len(data)

21047

In [ ]:
finaldata = data[["Title", "Plot"]]          # Required columns - Title and movie plot
finaldata = finaldata.set_index('Title')    # Setting the movie title as index
  

In [ ]:
finaldata.head(10)


,Plot
Title,
Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr..."
Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov..."
The Martyred Presidents,"The film, just over a minute long, is composed..."
"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...
Jack and the Beanstalk,The earliest known adaptation of the classic f...
Alice in Wonderland,"Alice follows a large white rabbit down a ""Rab..."
The Great Train Robbery,The film opens with two bandits breaking into ...
The Suburbanite,The film is about a family who move to the sub...
The Little Train Robbery,The opening scene shows the interior of the ro...


In [ ]:
finaldata["Plot"][0]

"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
  
  
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
  
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
  
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

def preprocess_sentences(text):
  text = text.lower()
  temp_sent =[]
  words = nltk.word_tokenize(text)
  tags = nltk.pos_tag(words)
  for i, word in enumerate(words):
      if tags[i][1] in VERB_CODES: 
          lemmatized = lemmatizer.lemmatize(word, 'v')
      else:
          lemmatized = lemmatizer.lemmatize(word)
      if lemmatized not in stop_words and lemmatized.isalpha():
          temp_sent.append(lemmatized)
          
  finalsent = ' '.join(temp_sent)
  finalsent = finalsent.replace("n't", " not")
  finalsent = finalsent.replace("'m", " am")
  finalsent = finalsent.replace("'s", " is")
  finalsent = finalsent.replace("'re", " are")
  finalsent = finalsent.replace("'ll", " will")
  finalsent = finalsent.replace("'ve", " have")
  finalsent = finalsent.replace("'d", " would")
  return finalsent
  
finaldata["plot_processed"]= finaldata["Plot"].apply(preprocess_sentences)
finaldata.head()

,Plot,plot_processed
Title,,
Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",bartender work saloon serve drink customer fil...
Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",moon paint smile face hang park night young co...
The Martyred Presidents,"The film, just over a minute long, is composed...",film minute long compose two shot first girl s...
"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,last second consist two shot first shot set wo...
Jack and the Beanstalk,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
  

tfidfvec = TfidfVectorizer()
tfidf_movieid = tfidfvec.fit_transform((finaldata["plot_processed"]))
  



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_movieid, tfidf_movieid)

In [ ]:
indices = pd.Series(finaldata.index)

In [ ]:
def recommendations(title, cosine_sim = cos_sim):
    recommended_movies = []
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_10_movies = list(similarity_scores.iloc[1:11].index)
    for i in top_10_movies:
        recommended_movies.append(list(finaldata.index)[i])
    return recommended_movies

In [ ]:
recommendations("Harry Potter and the Chamber of Secrets")

["Harry Potter and the Sorcerer's Stone",
 "Harry Potter and the Philosopher's Stone",
 'Harry Potter and the Deathly Hallows: Part I',
 'Harry Potter and the Deathly Hallows: Part 1',
 'Harry Potter and the Half-Blood Prince',
 'Harry Potter and the Deathly Hallows: Part II',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Harry Potter and the Order of the Phoenix',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban']

In [ ]:
recommendations("The Beast with Five Fingers")

['Checkpoint',
 'Blackmail',
 'Odds Against Tomorrow',
 'Suture',
 'Mitchell',
 'High Flyers',
 'Ordinary People',
 'Fruitvale Station',
 'The Exile',
 'Alien Origin']

In [ ]:
recommendations("Alien Origin")

['Hospital Massacre',
 'The Pretender',
 'Chasing Amy',
 'Night of the Demon',
 'The Good Girl',
 'Open Water',
 'Deep End',
 'Wise Girl',
 'Spy',
 'Shatter Dead']

In [ ]:
recommendations("The Martyred Presidents")

['Rancho Notorious',
 'One Eight Seven',
 "Other People's Money",
 'Terrible Teddy, the Grizzly King',
 'Abraham Lincoln: Vampire Hunter',
 'Saving Lincoln',
 'Abraham Lincoln',
 'Garfield: The Movie',
 'Garfield: A Tail of Two Kitties',
 'The Island']

In [ ]:
recommendations('Jack and the Beanstalk')

['Jack and the Beanstalk',
 'Traces of Red',
 'The Ruling Class',
 'Jack the Giant Slayer',
 'A Thousand Words',
 ' The Family Man',
 'Little Big Man',
 'Delirious',
 "The Pasha's Daughter",
 'Vacation in Reno']